In [28]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import geopandas as gp
import re

import warnings
warnings.filterwarnings('ignore')

In [29]:
# Setting up directories

WORKINGDIR = Path(os.getcwd())
PROJECTROOT = WORKINGDIR.parents[1]

POVERTY_ESTIMATES_DIR = Path(PROJECTROOT, "data", "gathered-datasets", "poverty-estimates.csv")
COMPLETE_DATASET = Path(PROJECTROOT, "data", "geolocation","ph_cities_joined_v2", "ph_cities_v2.shp")
OUTPUT_DATASET = Path(PROJECTROOT, "data", "gathered-datasets", "poverty-estimates-cleaned.csv")

In [30]:
ref_df = gp.read_file(COMPLETE_DATASET)
df = pd.read_csv(POVERTY_ESTIMATES_DIR)

In [31]:
ref_df.head()

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry
0,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168","MULTIPOLYGON (((118.57998 9.37215, 118.57982 9..."
1,1705101000,Abra De Ilog,Abra De Ilog,Occidental Mindoro,"abradeilog, occidentalmindoro",120.726826,13.443721,"13.4437209, 120.7268262","POLYGON ((120.60896 13.35233, 120.60797 13.373..."
2,300801000,Abucay,Abucay,Bataan,"abucay, bataan",120.534870,14.721315,"14.7213146, 120.5348704","POLYGON ((120.45676 14.69671, 120.45620 14.696..."
3,201501000,Abulug,Abulug,Cagayan,"abulug, cagayan",121.457273,18.443485,"18.4434854, 121.4572732","MULTIPOLYGON (((121.40276 18.40896, 121.40276 ..."
4,803701000,Abuyog,Abuyog,Leyte,"abuyog, leyte",125.011485,10.747102,"10.747102, 125.0114853","POLYGON ((125.04650 10.56751, 125.04588 10.576..."


In [32]:
keep_cols = ["name", "clean_idx", "province", "city_munic"]
ref_df = ref_df[keep_cols]

ref_df.head()

,name,clean_idx,province,city_munic
0,Aborlan,"aborlan, palawan",Palawan,Aborlan
1,Abra De Ilog,"abradeilog, occidentalmindoro",Occidental Mindoro,Abra De Ilog
2,Abucay,"abucay, bataan",Bataan,Abucay
3,Abulug,"abulug, cagayan",Cagayan,Abulug
4,Abuyog,"abuyog, leyte",Leyte,Abuyog


In [33]:
df.head()

,psgc,province,municipality-city,2009-poverty-inc,2012-poverty-inc,2015-poverty-inc,2009-cov,2012-cov,2015-cov,2009-conf-int-lowerlimit,2009-conf-int-upper-limit,2012-conf-int-lowerlimit,2012-conf-int-upper-limit,2015-conf-int-lowerlimit,2015-conf-int-upper-limit
0,133901,Metropolitan Manila,Tondo,"2,9","3,1","5,6","27,5","20,4","10,7","1,6","4,2","2,1","4,2","4,6","6,6"
1,133902,Metropolitan Manila,Binondo,"1,0","1,5","3,5","68,0","41,9","22,8","0,0","2,2","0,5","2,5","2,2","4,8"
2,133903,Metropolitan Manila,Quiapo,"2,1","2,1","7,1","42,4","28,1","17,9","0,6","3,5","1,1","3,1","5,0","9,2"
3,133904,Metropolitan Manila,San Nicolas,"2,4","3,0","6,7","57,0","53,6","27,2","0,1","4,7","0,4","5,5","3,7","9,7"
4,133905,Metropolitan Manila,Santa Cruz,"1,7","1,5","4,7","32,5","19,5","12,7","0,8","2,5","1,0","2,0","3,7","5,7"


In [34]:
def data_cleaner(text):
    # Make lower caps
    text = text.lower()

    # Remove "city"
    text = text.replace("city", "")

    # Remove " of "
    text = text.replace(" of ", "")
    
    # Remove all strings within parentheses
    text = text.split(" (")[0]
    
    # Remove (Capital)
    text = text.replace(" (capital)", "")

    # Remove punctuation marks
    text = re.sub(r"[^\w\s]", "", text)

    # Remove spaces
    text = text.replace(" ", "")

    # Replace compostela valley with Davao de Oro
    text = text.replace("compostelavalley", "davaodeoro")

    # Replace Maguindanao with del Norte/del Sur
    if "maguindanao" in text:
        text = "maguindanao"

    # Replace santo with sto
    text = text.replace("santo", "sto")

    # Replace santa with sta
    text = text.replace("santa", "sta")



    return text

In [35]:
def correct_province(province):
    if province == "North Cotabato":
        return "Cotabato"
    elif province == "Samar (Western Samar)":
        return "Samar"
    else:
        return province

In [36]:
def correct_index(index):
    index = index.replace('divilican, isabela',"divilacan, isabela")
    index = index.replace("cobarronguis, quirino", "cabarroguis, quirino")
    index = index.replace("sanidelfonso, bulacan", "sanildefonso, bulacan")
    index = index.replace("muñoz, nuevaecija", "sciencemuñoz, nuevaecija")
    index = index.replace("belizon, antique", "belison, antique")
    index = index.replace("prescarlospgarcia, bohol", "presidentcarlospgarcia, bohol")
    index = index.replace("cordoba, cebu", "cordova, cebu")
    index = index.replace("pinamungahan, cebu", "pinamungajan, cebu")
    index = index.replace("ozamis, misamisoccidental", "ozamiz, misamisoccidental")
    index = index.replace("koronodal, southcotabato", "koronadal, southcotabato")
    index = index.replace("bumbaran, lanaodelsur", "amaimanabilang, lanaodelsur")

    return index

In [37]:
df["province"] = df["province"].apply(correct_province)

In [38]:
"Cotabato" in list(df["province"].unique())

True

In [39]:
df["clean_province"] = (
    df["province"].apply(data_cleaner).apply(correct_province)
)

df["clean_city"] = df["municipality-city"].apply(
    data_cleaner
)

In [40]:
df.head()

,psgc,province,municipality-city,2009-poverty-inc,2012-poverty-inc,2015-poverty-inc,2009-cov,2012-cov,2015-cov,2009-conf-int-lowerlimit,2009-conf-int-upper-limit,2012-conf-int-lowerlimit,2012-conf-int-upper-limit,2015-conf-int-lowerlimit,2015-conf-int-upper-limit,clean_province,clean_city
0,133901,Metropolitan Manila,Tondo,"2,9","3,1","5,6","27,5","20,4","10,7","1,6","4,2","2,1","4,2","4,6","6,6",metropolitanmanila,tondo
1,133902,Metropolitan Manila,Binondo,"1,0","1,5","3,5","68,0","41,9","22,8","0,0","2,2","0,5","2,5","2,2","4,8",metropolitanmanila,binondo
2,133903,Metropolitan Manila,Quiapo,"2,1","2,1","7,1","42,4","28,1","17,9","0,6","3,5","1,1","3,1","5,0","9,2",metropolitanmanila,quiapo
3,133904,Metropolitan Manila,San Nicolas,"2,4","3,0","6,7","57,0","53,6","27,2","0,1","4,7","0,4","5,5","3,7","9,7",metropolitanmanila,sannicolas
4,133905,Metropolitan Manila,Santa Cruz,"1,7","1,5","4,7","32,5","19,5","12,7","0,8","2,5","1,0","2,0","3,7","5,7",metropolitanmanila,stacruz


In [41]:
df["clean_idx"] = (
    df["clean_city"].astype(str)
    + ", "
    + df["clean_province"].astype(str)
)

In [42]:
df["clean_idx"] = correct_index(df["clean_idx"])

In [43]:
df.head()

,psgc,province,municipality-city,2009-poverty-inc,2012-poverty-inc,2015-poverty-inc,2009-cov,2012-cov,2015-cov,2009-conf-int-lowerlimit,2009-conf-int-upper-limit,2012-conf-int-lowerlimit,2012-conf-int-upper-limit,2015-conf-int-lowerlimit,2015-conf-int-upper-limit,clean_province,clean_city,clean_idx
0,133901,Metropolitan Manila,Tondo,"2,9","3,1","5,6","27,5","20,4","10,7","1,6","4,2","2,1","4,2","4,6","6,6",metropolitanmanila,tondo,"tondo, metropolitanmanila"
1,133902,Metropolitan Manila,Binondo,"1,0","1,5","3,5","68,0","41,9","22,8","0,0","2,2","0,5","2,5","2,2","4,8",metropolitanmanila,binondo,"binondo, metropolitanmanila"
2,133903,Metropolitan Manila,Quiapo,"2,1","2,1","7,1","42,4","28,1","17,9","0,6","3,5","1,1","3,1","5,0","9,2",metropolitanmanila,quiapo,"quiapo, metropolitanmanila"
3,133904,Metropolitan Manila,San Nicolas,"2,4","3,0","6,7","57,0","53,6","27,2","0,1","4,7","0,4","5,5","3,7","9,7",metropolitanmanila,sannicolas,"sannicolas, metropolitanmanila"
4,133905,Metropolitan Manila,Santa Cruz,"1,7","1,5","4,7","32,5","19,5","12,7","0,8","2,5","1,0","2,0","3,7","5,7",metropolitanmanila,stacruz,"stacruz, metropolitanmanila"


In [44]:
(df["province"]=="North Cotabato").sum()

0

In [45]:
ref_idx= list(ref_df["clean_idx"].unique())

In [46]:
df_idx = list(df["clean_idx"].unique())

In [47]:
not_in_ref = [name for name in df_idx if name not in ref_idx]
not_in_ref

['tondo, metropolitanmanila',
 'binondo, metropolitanmanila',
 'quiapo, metropolitanmanila',
 'sannicolas, metropolitanmanila',
 'stacruz, metropolitanmanila',
 'sampaloc, metropolitanmanila',
 'sanmiguel, metropolitanmanila',
 'ermita, metropolitanmanila',
 'intramuros, metropolitanmanila',
 'malate, metropolitanmanila',
 'paco, metropolitanmanila',
 'pandacan, metropolitanmanila',
 'portarea, metropolitanmanila',
 'staana, metropolitanmanila']

In [48]:
not_in_df = [name for name in ref_idx if name not in df_idx]
not_in_df

['cotabato, maguindanao',
 'isabela, basilan',
 'kalayaan, palawan',
 'manila, metropolitanmanila']

In [49]:
df = ref_df.merge(df, how="left", on="clean_idx")

In [50]:
df.head()

,name,clean_idx,province_x,city_munic,psgc,province_y,municipality-city,2009-poverty-inc,2012-poverty-inc,2015-poverty-inc,...,2012-cov,2015-cov,2009-conf-int-lowerlimit,2009-conf-int-upper-limit,2012-conf-int-lowerlimit,2012-conf-int-upper-limit,2015-conf-int-lowerlimit,2015-conf-int-upper-limit,clean_province,clean_city
0,Aborlan,"aborlan, palawan",Palawan,Aborlan,175301.0,Palawan,Aborlan,"23,4","22,4","21,6",...,"12,4","11,8","16,6","30,2","17,8","26,9","17,4","25,7",palawan,aborlan
1,Abra De Ilog,"abradeilog, occidentalmindoro",Occidental Mindoro,Abra De Ilog,175101.0,Occidental Mindoro,Abra de Ilog,"37,5","32,3","34,8",...,"13,8","12,5","27,4","47,5","25,0","39,6","27,6","41,9",occidentalmindoro,abradeilog
2,Abucay,"abucay, bataan",Bataan,Abucay,30801.0,Bataan,Abucay,"7,0","4,0","14,5",...,"25,8","20,5","3,6","10,4","2,3","5,6","9,6","19,4",bataan,abucay
3,Abulug,"abulug, cagayan",Cagayan,Abulug,21501.0,Cagayan,Abulug,"21,6","20,9","14,4",...,"9,9","15,0","17,3","25,9","17,5","24,3","10,8","17,9",cagayan,abulug
4,Abuyog,"abuyog, leyte",Leyte,Abuyog,83701.0,Leyte,Abuyog,"35,4","35,9","36,1",...,"6,2","7,7","32,1","38,6","32,3","39,6","31,5","40,6",leyte,abuyog


In [51]:
years = ["2009", "2012", "2015"]

df_list = []

for year in years: 
    col_years = [col for col in df.columns if year in col]
    df_year = df[["name"]+col_years]
    df_year["year"] = str(year)
    
    rename_dict = {}
    for col in col_years:
        rename_dict[col] = col[5:]
    
    df_year.rename(columns=rename_dict, inplace=True)
    df_list.append(df_year)

In [52]:
df_merged = pd.concat(df_list, axis=0)

In [53]:
df_merged.to_csv(OUTPUT_DATASET)
df_merged

,name,poverty-inc,cov,conf-int-lowerlimit,conf-int-upper-limit,year
0,Aborlan,"23,4","17,7","16,6","30,2",2009
1,Abra De Ilog,"37,5","16,4","27,4","47,5",2009
2,Abucay,"7,0","29,5","3,6","10,4",2009
3,Abulug,"21,6","12,1","17,3","25,9",2009
4,Abuyog,"35,4","5,5","32,1","38,6",2009
...,...,...,...,...,...,...
1629,Zamboanga,"17,3","9,0","14,7","19,8",2015
1630,Zamboanguita,"39,9","10,1","33,3","46,5",2015
1631,Zaragoza,"13,2","15,5","9,8","16,6",2015
1632,Zarraga,"17,4","14,8","13,2","21,6",2015
